4. Create a dual-axis chart comparing the average installs and revenue for free vs. 
paid apps within the top 3 app categories. Apply filters to exclude apps with fewer than 10,000 installs and 
revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and 
content rating should be Everyone and app name should not have more than 30 characters including space and special character .
this graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.


In [66]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz


In [68]:
data = pd.read_csv('Play Store Data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [54]:
filtered_data = data.copy()

In [70]:
# Step 1: Replace non-numeric values with '0' and convert to string for safe regex operations
filtered_data['Installs'] = filtered_data['Installs'].fillna('0').astype(str)

# Step 2: Remove non-numeric characters (commas, plus signs)
filtered_data['Installs'] = filtered_data['Installs'].replace(r'[\+,]', '', regex=True)

# Step 3: Convert to integers
filtered_data['Installs'] = pd.to_numeric(data['Installs'], errors='coerce').fillna(0).astype(int)


In [62]:
filtered_data['Price'] = filtered_data['Price'].str.replace('[^0-9]', '', regex=True)
filtered_data['Price'] = filtered_data['Price'].replace('', '0').astype(int)

In [106]:
# Check the result
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  float64
 5   Installs        10841 non-null  int32  
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  int32  
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10841 non-null  float64
 13  Multiplier      10841 non-null  float64
dtypes: float64(4), int32(2), object(8)
memory usage: 1.1+ MB


In [90]:
# Ensure all entries in 'Android Ver' column are strings
filtered_data['Android Ver'] = filtered_data['Android Ver'].astype(str)

# Extract numeric Android version (e.g., '4.0.3' from '4.0.3 and up') and handle missing values
filtered_data['Android Ver'] = (
    filtered_data['Android Ver']
    .str.extract(r'([0-9]+(?:\.[0-9]+)*)')  # Extract the version numbers
    .fillna('0')  # Replace missing values with '0'
    .astype(float)  # Convert to float
)


In [104]:
filtered_data['Size'] = filtered_data['Size'].astype(str)
# Step 2: Extract size multiplier (k or M), replace with corresponding numeric values, and handle missing cases
filtered_data['Multiplier'] = filtered_data['Size'].str.extract(r'[\d\.]+([kM])', expand=False)\
    .replace({'k': 1 / 1024, 'M': 1}).fillna(1).astype(float)
# Step 3: Remove non-numeric characters, convert to numeric, and multiply by the multiplier
filtered_data['Size'] = (filtered_data['Size'].str.replace(r'[^\d\.]', '', regex=True).replace('', '0').astype(float))

C:\Users\USER\AppData\Local\Temp\ipykernel_12768\1029575550.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'k': 1 / 1024, 'M': 1}).fillna(1).astype(float)


In [110]:
# Clean 'Android Version' column
#filtered_data['Android Ver'] = (filtered_data['Android Ver'].str(r'([0-9]+\\.[0-9]+(?:\\.[0-9]+)?)').fillna('0').astype(float))
# Clean 'Size' column: Convert sizes like '15M' or '100k' to MB and handle missing values
#filtered_data['Size'] = filtered_data['Size'].replace({'k': 'e-3', 'M': 'e3'}, regex=True)
#filtered_data['Size'] = filtered_data['Size'].replace('', '0').map(pd.eval).astype(float)

# Apply filters
filtered_data = filtered_data[
    (filtered_data['Installs'] >= 10000) &
    (filtered_data['Price'] >= 10000) &
    (filtered_data['Android Ver'] > 4.0) &
    (filtered_data['Size'] > 15) &
    (filtered_data['Content Rating'] == 'Everyone') &
    (filtered_data['App'].str.len() <= 30)
]


In [114]:
# Group by category and calculate average installs and revenue
grouped = filtered_data.groupby(['Category', 'Type']).agg(
    Avg_Installs=('Installs', 'mean'),
    Avg_Revenue=('Price', 'mean')
).reset_index()

# Identify top 3 categories by total installs
top_categories = (
    grouped.groupby('Category')['Avg_Installs']
    .sum()
    .nlargest(3)
    .index
)

# Filter for the top 3 categories
top_data = grouped[grouped['Category'].isin(top_categories)]

# Plot the dual-axis chart
def plot_chart():
    fig, ax1 = plt.subplots(figsize=(10, 6))

    for category in top_categories:
        category_data = top_data[top_data['Category'] == category]
        
        ax1.bar(
            category_data['Type'], 
            category_data['Avg_Installs'], 
            label=f'{category} - Installs', 
            alpha=0.7
        )

    ax2 = ax1.twinx()

    for category in top_categories:
        category_data = top_data[top_data['Category'] == category]
        
        ax2.plot(
            category_data['Type'], 
            category_data['Avg_Revenue'], 
            marker='o', 
            label=f'{category} - Revenue'
        )

    ax1.set_xlabel('App Type (Free vs Paid)')
    ax1.set_ylabel('Average Installs')
    ax2.set_ylabel('Average Revenue ($)')
    ax1.set_title('Average Installs and Revenue for Top 3 App Categories')
    
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()



In [116]:
# Check current time for IST
def is_time_valid():
    ist = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(ist).time()
    return current_time >= datetime.strptime("13:00", "%H:%M").time() and \
           current_time <= datetime.strptime("14:00", "%H:%M").time()

if is_time_valid():
    plot_chart()
else:
    print("The chart can only be displayed between 1 PM IST and 2 PM IST.")


The chart can only be displayed between 1 PM IST and 2 PM IST.
